<a href="https://colab.research.google.com/github/HwangHanJae/Recommendation_Study/blob/main/inflearn_recsys/CF_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이웃을 고려한 CF

이전에 구했던 CF_simple 모델은 이웃을 고려하지 않고 평점의 가중 평균을 구하여 성능을 확인했습니다.

CF_KNN은 이웃 혹은 임계값을 설정하여 조건에 부합하는 이웃만을 피어그룹으로 설정하여 평가를 진행합니다.

ex)
- 이웃 : 사용자가 설정하는 k 명
- 임계값 : 상관계수 혹은 코사인 유사도가 0.8 이상



# 데이터 읽기

무비렌즈의 유저의 정보(u.user) 읽기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#베이스 경로 설정
base = '/content/drive/MyDrive/RecoSys/Data'

# u.user 파일 경로 설정
u_user_path = os.path.join(base, 'u.user')

#필요한 컬럼 정의
u_cols = ['user_id','age','sex','occupation','zip_code']

#데이터 읽어오기
users = pd.read_csv(u_user_path, sep='|', names = u_cols, encoding='latin-1')
#users 데이터 프레임에 인덱스(user_id) 지정
users = users.set_index('user_id')

#상위 5개
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


무비렌즈의 영화의 정보(u.item) 읽기

In [2]:
#u.item의 파일 경로 설정
u_item_path = os.path.join(base, 'u.item')

#필요한 컬럼 정의
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action',
          'Adventure','Animation','Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy','Film-Noir','Horror','Musical', 'Mystery','Romance','Sci-Fi','Thriller','War','Western']

# 데이터 읽어오기
movies = pd.read_csv(u_item_path, sep='|',names =i_cols, encoding='latin-1')
# movies 데이터 프레임에 인덱스(movie_id) 지정
movies = movies.set_index('movie_id')

#상위 5개
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


무비렌즈의 평점 정보(u.data)읽기

In [3]:
#u.data의 파일경로 지정
u_data_path = os.path.join(base, 'u.data')

#필요한 컬럼 정의
r_cols = ['user_id', 'movie_id','rating','timestamp']

#데이터 읽어오기
ratings = pd.read_csv(u_data_path, sep='\t',names = r_cols, encoding='latin-1')

#상위 5개
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# 평가지표


In [4]:
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])

  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

# 모델링

In [6]:
# 데이터 셋 만들기
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, stratify=y)

#sparse matrix 만들기
ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [7]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


In [11]:
def CF_KNN(user_id, movie_id, neighbor_size=0):
  #train set안에 movie_id가 존재한다면
  if movie_id in ratings_matrix.columns:
    #해당 유저(user_id)와 다른 유저의 유사도를 구함
    sim_scores = user_similarity[user_id].copy()
    #주어진 영화(movie_id)의 평점정보를 가져옴
    #어떤 유저가 평점을 했는지 안하였는지 확인할 수 있음
    movie_ratings = ratings_matrix[movie_id].copy()
    #주어진 영화(movie_id)에 대하여 아직 평가를 하지 않은 유저정보(user_id)를 가져옴
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    #평가를 하지 않은 유저를 movie_ratings와 sim_scores에서 drop시킴
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    #이웃을 지정하지 않았다면 유사도와 영화의 평점을 가중평균하여 계산

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    
    else:
      # 이웃을 지정하고 sim_score 크기가 1보다 크다면
      if len(sim_scores) > 1:
        # 이웃의 크기를 sim_score의 사이즈나 neighbor_size의 크기 중 작은것으로 선택
        # 주어진 이웃의 크기가 20이지만 sim_score의 크기가 10일 수 있음 이럴 경우 10을 선택
        neighbor_size = min(neighbor_size, len(sim_scores))

        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        # sim_score를 유사도값 기준으로 오름차순으로 정렬하고 index를 뽑음
        user_idx = np.argsort(sim_scores)
        # 이웃의 크기만큼 내림차순으로 sim_scores, movie_ratings를 구함
        # 이웃의 크기만큼의 피어그룹이 생성되고 그에 따른 영화의 평점 정보를 받을 수 있음
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        # 피어그룹의 유사도와 영화의 평점을 가중평균하여 계산
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        # sim_score값이 0보다 작거나 같다면 
        # 평균값인 3.0을 지정
        mean_rating = 3.0
  else:
    #위와 동일
    mean_rating = 3.0
  
  return mean_rating


In [12]:
score(CF_KNN, neighbor_size = 30)

1.004118372849313

모델의 성능이 CF_Simple 모델 보다는 좋은 것으로 확인됩니다.

모델의 성능을 확인하였으니 실제 주어진 사용자에게 추천을 해보도록 하겠습니다.

In [15]:
# 모델의 성능을 평가할 때는 train, test세트로 분리해서 평가하였지만
# 실제 추천을 진행할 때는 그럴 필요가 없기 때문에 전체데이터를 활용
ratings_matrix = ratings.pivot_table(values = 'rating',
                                     index = 'user_id',
                                     columns = 'movie_id')
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)

In [26]:
def recom_movie(user_id, n_items, neighbor_size=30):
  # 입력받은 유저가 평가한 영화의 평점정보를 가져옵니다.
  # 영화를 평가했다면 평점이 평가하지 않았다면 NaN값이 나옵니다.
  user_movie = ratings_matrix.loc[user_id].copy()
  # 모든 영화를 확인하면서
  for movie in ratings_matrix.columns:
    # 입력받은 유저가 영화를 평가했다면 True, 하지않았다면 False를 반환
    # 입력받은 유저가 영화를 평가했다면 평점을 예측하여 추천할 필요가 없기 때문에 0으로 값을 입력함
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0
    else:
      # 입력받은 유저가 아직 평가하지 않은 영화에 대하여 평점을 예측하여 매김
      user_movie.loc[movie] = CF_KNN(user_id, movie, neighbor_size)
  
  # 모든 영화에 평점을 재 입력하였다면
  # 예측한 평점을 기준으로 내림차순 정렬
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]

  #영화 테이블에서 평점이 높은 영화의 인덱스를 가져와서
  #영화의 제목(title)만 추출
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']

  return recommendations

In [27]:
recom_movie(user_id=729, n_items=5, neighbor_size=30)

movie_id
1189                      Prefontaine (1997)
1293                         Star Kid (1997)
1467    Saint of Fort Washington, The (1993)
1500               Santa with Muscles (1996)
22                         Braveheart (1995)
Name: title, dtype: object